In [ ]:
import random
import gym
import pandas as pd 
import seaborn as sns
import numpy as np
import random
import torch
import torch.nn as nn
from tqdm import tqdm  # Progress bar
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from collections import deque
from sklearn.preprocessing import StandardScaler
import random
# Normalize features
from sklearn.preprocessing import StandardScaler
from collections import deque
import collections
from keras import backend as K
import tensorflow as tf
from gym import spaces
import matplotlib.pyplot as plt
K.clear_session()
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/eqqerff/preprocessed.csv')
data.drop(columns= ['Unnamed: 0','index'], inplace = True)
# preprocessing
data['DATETIME'] = pd.to_datetime(data['DATETIME'])
data['TIME'] = data['TIME'].astype(str).str.rstrip('000')
data['hour'] = data['DATETIME'].dt.hour
data['minute'] = data['DATETIME'].dt.minute
data['day_of_week'] = data['DATETIME'].dt.day_name()
data['second'] =  data['DATETIME'].dt.second
day_map = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}
data['day_of_week'] = data['day_of_week'].map(day_map)
data.drop(columns= ['DATETIME','TIME'], inplace = True) 
data.drop(columns= ['TRADE_PRICE','TRADENO'], inplace = True)
data= data.dropna()
data

In [ ]:
def reward_function(df):
    # Calculate best bid-ask spread for each date
    best_spread_per_date = df.groupby('DATE').apply(lambda x: x['BID_PRICE'].max() - x['ASK_PRICE'].min())

    rewards = []
    for index, row in df.iterrows():
        # Calculate execution price deviation from best bid-ask spread
        execution_price_deviation = row['PRICE'] - best_spread_per_date[row['DATE']]

        # Calculate execution speed penalty
        execution_speed_penalty = 100 / 1000  # converting milliseconds to seconds

        # Calculate commission cost
        commission_cost = 0.01 * row['TRADE_VOLUME']
        # Calculate market impact penalty (quadratic function)
        market_impact_penalty = 0.0001 * (row['TRADE_VOLUME'] ** 2)

        # Calculate total reward
        reward = -execution_price_deviation - execution_speed_penalty -  commission_cost- market_impact_penalty
        rewards.append(reward)

    return rewards

rewards = reward_function(data)

In [ ]:
from tqdm import tqdm
# Normalizing the data
def normalize_data(df):
    df_normalized = df.copy()
    for column in df.columns:
        if df[column].dtype == np.float64 or df[column].dtype == np.int64:
            df_normalized[column] = (df[column] - df[column].mean()) / df[column].std()
    return df_normalized

data = normalize_data(data)
data.drop(columns = ['DATE', 'ACTION','ORDERNO','TRADE_TIME'], inplace = True)
# Create a dictionary to map string values to numeric values
buysell_map = {'B': 0, 'S': 1}

# Map the string values to numeric values
data['BUYSELL'] = data['BUYSELL'].map(buysell_map)
data.drop(columns = 'TRADEPRICE', inplace = True)

In [ ]:
data.columns

In [ ]:
# Ensure no missing values remain in the dataset
data.fillna(0, inplace=True)

# Normalize data function using StandardScaler
def normalize_data(df):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df)
    return pd.DataFrame(scaled_data, columns=df.columns)

# Neural Network architecture
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Experience Replay
class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)
        
    def push(self, transition):
        self.memory.append(transition)
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

# Double DQN Agent
class DQNAgent:
    def __init__(self, state_dim, action_dim, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995, gamma=0.99, lr=0.001, replay_memory_size=10000, batch_size=32, target_update=10):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.gamma = gamma
        self.lr = lr
        self.batch_size = batch_size
        self.target_update = target_update
        self.memory = ReplayMemory(replay_memory_size)
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.policy_net = DQN(state_dim, action_dim).to(self.device)
        self.target_net = DQN(state_dim, action_dim).to(self.device)
        self.optimizer = optim.RMSprop(self.policy_net.parameters(), lr=lr)
        self.criterion = nn.MSELoss()
        
        self.update_target_network()

    def update_target_network(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()
        
    def select_action(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(0, self.action_dim)
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        with torch.no_grad():
            return self.policy_net(state).argmax().item()
        
    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        
        minibatch = self.memory.sample(self.batch_size)
        states, actions, rewards, next_states, dones = zip(*minibatch)
        
        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device).unsqueeze(1)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)
        
        current_q_values = self.policy_net(states).gather(1, actions)
        next_actions = self.policy_net(next_states).argmax(dim=1, keepdim=True)
        next_q_values = self.target_net(next_states).gather(1, next_actions).squeeze(1)
        expected_q_values = rewards + (self.gamma * next_q_values * (1 - dones))
        
        loss = self.criterion(current_q_values, expected_q_values.unsqueeze(1))
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def remember(self, state, action, reward, next_state, done):
        self.memory.push((state, action, reward, next_state, done))

# Environment for Optimal Execution
class OptimalExecutionEnv:
    def __init__(self, data):
        self.data = data
        self.n = len(data)
        self.current_step = 0
        
    def reset(self):
        self.current_step = 0
        return self.data.iloc[self.current_step].values
        
    def step(self, action):
        state = self.data.iloc[self.current_step].values
        next_state = self.data.iloc[self.current_step + 1].values if self.current_step < self.n - 1 else state
        reward = self.calculate_reward(self.data.iloc[self.current_step], action)
        done = self.current_step == self.n - 1
        self.current_step += 1
        return next_state, reward, done
        
    def calculate_reward(self, row, action):
        execution_price_deviation = row['PRICE'] - row['BID_ASK_SPREAD']
        execution_speed_penalty = 100 / 1000
        commission_cost = 0.01 * row['VOLUME']
        market_impact_penalty = 0.0001 * (row['VOLUME'] ** 2)
        reward = -execution_price_deviation - execution_speed_penalty - commission_cost - market_impact_penalty
        return reward

# Main training loop
def train(agent, env, episodes):
    for e in tqdm(range(episodes), desc="Training Episodes"):
        state = env.reset()
        total_rewards = 0
        while True:
            action = agent.select_action(state)
            next_state, reward, done = env.step(action)
            total_rewards += reward
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print(f"Episode {e+1}/{episodes} - Total reward: {total_rewards}, Epsilon: {agent.epsilon}")
                break
            agent.replay()
        if e % agent.target_update == 0:
            agent.update_target_network()

# Data Preprocessing
data = normalize_data(data)
state_dim = data.shape[1]
action_dim = 2  # Buy or Sell

# Initialize environment and agent
env = OptimalExecutionEnv(data)
agent = DQNAgent(state_dim=state_dim, action_dim=action_dim)

# Train the agent
train(agent, env, episodes=100)

# Evaluate the agent
def evaluate_agent(agent, env, episodes=10):
    total_rewards = []
    for _ in range(episodes):
        state = env.reset()
        total_reward = 0
        while True:
            action = agent.select_action(state)
            next_state, reward, done = env.step(action)
            total_reward += reward
            state = next_state
            if done:
                total_rewards.append(total_reward)
                break
    average_reward = np.mean(total_rewards)
    print(f"Average Reward over {episodes} episodes: {average_reward}")

evaluate_agent(agent, env, episodes=10)


In [ ]:
# Calculate statistics
def calculate_statistics(rewards):
    avg_reward = np.mean(rewards)
    max_reward = np.max(rewards)
    min_reward = np.min(rewards)
    std_reward = np.std(rewards)
    return avg_reward, max_reward, min_reward, std_reward

# Plot training performance
def plot_training_performance(rewards):
    plt.figure(figsize=(10, 6))
    plt.plot(rewards)
    plt.title("Training Performance")
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    plt.grid(True)
    plt.show()

# Plot Optimal Actions for All Possible States
def plot_optimal_actions(agent, env):
    states = np.array([[t, i, p] for t in range(env.n) for i in range(agent.state_dim) for p in range(agent.state_dim)])
    actions = [agent.select_action(state) for state in states]
    actions = np.array(actions).reshape((env.n, agent.state_dim, agent.state_dim))
    
    plt.figure(figsize=(15, 10))
    for i in range(agent.state_dim):
        for j in range(agent.state_dim):
            plt.subplot(agent.state_dim, agent.state_dim, i * agent.state_dim + j + 1)
            sns.heatmap(actions[:, i, j], cmap="YlGnBu", cbar=False, xticklabels=False, yticklabels=False)
            plt.title(f"Price: {i}, Inventory: {j}")
            plt.xlabel("Time")
            plt.ylabel("Inventory Remaining")
    plt.tight_layout()
    plt.show()

# Train the agent
rewards = train(agent, env, episodes=100)

# Calculate statistics
avg_reward, max_reward, min_reward, std_reward = calculate_statistics(rewards)
print(f"Average Reward: {avg_reward}")
print(f"Max Reward: {max_reward}")
print(f"Min Reward: {min_reward}")
print(f"Standard Deviation of Reward: {std_reward}")

# Plot training performance
plot_training_performance(rewards)

# Plot Optimal Actions for All Possible States
plot_optimal_actions(agent, env)